### Things to do

##### scrap data from scorecard and add more data
##### last performaces between two teams
##### multicollinearity
##### Feature Selection and PCA
##### No SMOTE : class balanced


In [2]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from datetime import datetime
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
label_encoder = preprocessing.LabelEncoder()

teams_2021 = ["afghanistan","australia","bangladesh","england","india","ireland","namibia","netherlands","new zealand",\
             "oman","pakistan","papua new guinea","scotland","sri lanka","west indies"]

def get_matches_played(df, row, team):
    matches_played = df[((df.date < row.date))&((df.team1 == team)|(df.team2 == team))].shape[0]
    return matches_played if matches_played else 1

def get_matches_won(df, row, team):
    matches_won = df[((df.date < row.date))&(df.winner == team)].shape[0]
    return matches_won if matches_won else 0

def get_past_data(df, row, team, number):
    tempdf = df[((df.date < row.date))&((df.team1 == team)|(df.team2 == team))].tail(number)
    total = tempdf.shape[0]
    wins = tempdf[tempdf.winner == team].shape[0]
    return wins/total if total>=5 else np.nan

In [17]:
#target variable would be winner team1 or team2
df=pd.DataFrame()
for year in range(2005,2021+1,1):
    df = pd.concat([df,pd.read_csv(r"C:\Users\Alok\Downloads\dataset\cricket\T20_Data_%s.csv"%year)])

#loading ipl data
ipldf = pd.read_csv(r"C:\Users\Alok\Downloads\dataset\cricket\ipl_data.csv") #IPL data

#concatinating both data
df = pd.concat([df,ipldf])

df.reset_index(drop=True)

# converting date into datetime format for dates comparison
df.date = pd.to_datetime(df.date)

df = df.sort_values(by='date')
#removing non-resulting matches, as non resulting match doesn't contribute much
df = df[df.winner != "draw"]

#converting "team" name into target variable which is team1 or team2
df['team'] = np.where(df.winner == df.team1,0,1)

# create mactches played column, winning percentages, losing percentages for both teams, last 10 matches (win,lose,win/lose) ratio

# deriving new features below
teams = {'t1':"team1",'t2':"team2"}
for team in teams.keys():
    df['%s_matches_played'%team] = df.apply(lambda row: get_matches_played(df, row, row[teams[team]]), axis=1)
    df['%s_matches_won'%team] = df.apply(lambda row: get_matches_won(df, row, row[teams[team]]), axis=1)
    df['%s_matches_lost'%team] = df['%s_matches_played'%team] - df['%s_matches_won'%team]
    df['%s_win_per'%team] = df['%s_matches_won'%team]/df['%s_matches_played'%team]
    df['%s_odds_of_win'%team] = df['%s_matches_won'%team]/df['%s_matches_lost'%team]
    df['%s_win_prob'%team] = df['%s_odds_of_win'%team]/(1+df['%s_odds_of_win'%team]) #this is drived probability will be used in model training
    df['%s_past10_win_per'%team] = df.apply(lambda row: get_past_data(df, row, row[teams[team]],10),axis=1) #past performance
    df['%s_past10_lose_per'%team] = 1-df['%s_past10_win_per'%team]

#resetting index
df=df.reset_index(drop=True)

# making a copy to keep original data safe which will be used earlier
df_ex = df.copy()
# missing value treatment
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(0.0,inplace=True)

#encoding team names
#df['encode_team1']= label_encoder.fit_transform(df['team1'])
#df['encode_team2']= label_encoder.fit_transform(df['team2']) 

#taking target column out
target = df[['team']]

# dropping unneeded columns
df.drop(["date","countries","result","winner","team","team1","team2"],axis=1,inplace=True)

#test train split
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size = 0.05, random_state = 67) #xgb rs=43,size=0.05

#defining a model
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
pred_result = xgb.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test,pred_result).ravel()
print("Accuracy : %s"%((tn+tp)/(tn+tp+fn+fp)))

C:\Users\Alok\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Alok\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Alok\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to 

[11:34:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy : 0.514018691588785


In [18]:
pd.read_csv(r"C:\Users\Alok\Downloads\dataset\cricket\ipl_data.csv") #IPL data

,date,countries,result,team1,team2,winner
0,18-04-2008,bangalore v. kolkata,kolkata by 140 runs,bangalore,kolkata,kolkata
1,19-04-2008,punjab v. chennai,chennai by 33 runs,punjab,chennai,chennai
2,19-04-2008,delhi v. rajasthan,delhi by 9 wickets,delhi,rajasthan,delhi
3,20-04-2008,kolkata v. deccan,kolkata by 5 wickets,kolkata,hyderabad,kolkata
4,20-04-2008,mumbai v. bangalore,bangalore by 5 wickets,mumbai,bangalore,bangalore
...,...,...,...,...,...,...
877,07-10-2021,kolkata v. rajasthan,kolkata by 86 runs,kolkata,rajasthan,kolkata
878,08-10-2021,mumbai v. hyderabad,mumbai by 42 runs,mumbai,hyderabad,mumbai
879,08-10-2021,delhi v. bangalore,bangalore by 7 wickets,delhi,bangalore,bangalore
880,10-10-2021,chennai v. delhi,chennai by 4 wickets,chennai,delhi,chennai
